In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from google.colab import drive
from tensorflow.keras.layers import Flatten, Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam


In [ ]:
# Mount Google Drive (if dataset is stored there)
drive.mount('/content/drive')

# Paths to your dataset (adjust according to your dataset path in Google Drive)
train_dir = '/content/drive/MyDrive/train'
test_dir = '/content/drive/MyDrive/test'

Mounted at /content/drive


In [ ]:
# Image data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Loading the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # ResNet50 expects 224x224 input images
    batch_size=32,
    class_mode='binary'  # Assuming binary classification (covid vs normal)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)



Found 2754 images belonging to 2 classes.
Found 1288 images belonging to 2 classes.


In [ ]:
# Loading the pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Adding custom classification layers on top of ResNet50
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Calculate steps_per_epoch and validation_steps based on dataset size
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=test_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('/content/resnet50_covid19_classifier.h5')

# Optionally: evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


86/86 ━━━━━━━━━━━━━━━━━━━━ 1056s 12s/step - accuracy: 0.7837 - loss: 1.4388 - val_accuracy: 0.9102 - val_loss: 0.2985
Epoch 2/10
 1/86 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.9375 - loss: 0.2932

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


86/86 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - accuracy: 0.9375 - loss: 0.2932 - val_accuracy: 0.8750 - val_loss: 0.2980
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.8424 - loss: 0.4609 - val_accuracy: 0.9102 - val_loss: 0.3528
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7812 - loss: 0.4409 - val_accuracy: 0.8750 - val_loss: 0.3738
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.8429 - loss: 0.4469 - val_accuracy: 0.9102 - val_loss: 0.2760
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8125 - loss: 0.4383 - val_accuracy: 0.8750 - val_loss: 0.3135
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.8576 - loss: 0.4184 - val_accuracy: 0.9094 - val_loss: 0.2791
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - accuracy: 0.7812 - loss: 0.5573 - val_accuracy: 1.0000 - val_loss: 0.1970
Epoch 9/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.8535 - loss: 0.4374 - val_accuracy: 0.9102 - val_loss: 0.241

41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 446ms/step - accuracy: 0.9141 - loss: 0.2312
Test Accuracy: 90.99%


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Adding custom classification layers on top of ResNet50 with Leaky ReLU
x = base_model.output
x = Flatten()(x)
x = Dense(256)(x)
x = LeakyReLU(alpha=0.01)(x)  # Leaky ReLU activation with alpha parameter
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model using Adam as the optimizer
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Calculate steps_per_epoch and validation_steps based on dataset size
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=test_generator,
    validation_steps=validation_steps
)

# Save the model
model.save('/content/resnet50_covid19_classifier.h5')

# Optionally: evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


86/86 ━━━━━━━━━━━━━━━━━━━━ 1523s 17s/step - accuracy: 0.7605 - loss: 3.7906 - val_accuracy: 0.9094 - val_loss: 0.2969
Epoch 2/10
 1/86 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - accuracy: 0.9062 - loss: 0.3699

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


86/86 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.9062 - loss: 0.3699 - val_accuracy: 1.0000 - val_loss: 0.1309
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.8426 - loss: 0.4353 - val_accuracy: 0.9094 - val_loss: 0.2794
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step - accuracy: 0.8438 - loss: 0.4231 - val_accuracy: 1.0000 - val_loss: 0.0939
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.8350 - loss: 0.4511 - val_accuracy: 0.9109 - val_loss: 0.2687
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.8438 - loss: 0.4480 - val_accuracy: 0.7500 - val_loss: 0.5438
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.8360 - loss: 0.4396 - val_accuracy: 0.9102 - val_loss: 0.2928
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.8125 - loss: 0.3940 - val_accuracy: 0.8750 - val_loss: 0.3602
Epoch 9/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.8377 - loss: 0.4101 - val_accuracy: 0.9102 - val_loss: 0.

41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 456ms/step - accuracy: 0.9180 - loss: 0.2569
Test Accuracy: 90.99%
